In [1]:
import pandas as pd
import numpy as np
import re

import spacy
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [2]:
file_path = '/kaggle/input/sms-spam-collection-a-more-diverse-dataset/train.csv'

df = pd.read_csv(file_path)

df.head(10)

,sms,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...\n,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5574 entries, 0 to 5573
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sms     5574 non-null   object
 1   label   5574 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 87.2+ KB


In [4]:
df.isnull().sum()

sms      0
label    0
dtype: int64

In [5]:
# 1. Очистка тексту за допомогою spaCy та NLTK
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words("english"))

def clean_text(text):
    # Видалення небажаних символів
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Залишаємо лише літери та пробіли
    text = text.lower()  # Перетворення в нижній регістр
    # Токенізація та видалення стоп-слів
    doc = nlp(text)
    #Лемматизація перетворює слова до базової форми (наприклад, "running" → "run"). 
    #Видаляються слова зі списку стоп-слів і пунктуація.
    tokens = [token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct]
    #Об’єднання очищених токенів у текст
    return " ".join(tokens)

df['sms_cleaned'] = df['sms'].apply(clean_text)



In [6]:
# 2. Перетворення тексту на числові ознаки (TF-IDF)
#Після очистки текст потрібно перетворити на формат, придатний для моделі машинного навчання.
#Для цього використовується метод TF-IDF
#Цей метод перетворює текст на числовий вектор, що відображає важливість кожного слова в тексті відносно всіх документів

vectorizer = TfidfVectorizer(max_features=500)
X = vectorizer.fit_transform(df['sms_cleaned']).toarray()

#Текст очищеного стовпця sms_cleaned перетворюється на матрицю чисел.
#Кожен рядок матриці представляє одне повідомлення, а кожен стовпець — слово.
#Результат: X — це матриця розміру (кількість повідомлень × 500).
y = df['label']

In [7]:
# 3. Розділення даних на тренувальну і тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Модель логістичної регресії
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Прогнозування
y_pred = model.predict(X_test)

# Оцінка моделі
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Вивід підготовлених даних
print(df[['sms', 'sms_cleaned']])

Accuracy: 0.9721973094170404
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98       954
           1       0.96      0.84      0.90       161

    accuracy                           0.97      1115
   macro avg       0.97      0.92      0.94      1115
weighted avg       0.97      0.97      0.97      1115

                                                    sms  \
0     Go until jurong point, crazy.. Available only ...   
1                       Ok lar... Joking wif u oni...\n   
2     Free entry in 2 a wkly comp to win FA Cup fina...   
3     U dun say so early hor... U c already then say...   
4     Nah I don't think he goes to usf, he lives aro...   
...                                                 ...   
5569  This is the 2nd time we have tried 2 contact u...   
5570             Will ü b going to esplanade fr home?\n   
5571  Pity, * was in mood for that. So...any other s...   
5572  The guy did some bitchin